# Sentiment Analysis of Description of Twitter Profiles

In [1]:
# Import Libraries and Data
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline, Trainer, TrainingArguments
from datasets import Dataset



# Load the data
df = pd.read_csv("twitter_human_bots_dataset.csv")
df.head()


/Users/eldricksim/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/eldricksim/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Unnamed: 0,created_at,default_profile,default_profile_image,description,favourites_count,followers_count,friends_count,geo_enabled,id,lang,location,profile_background_image_url,profile_image_url,screen_name,statuses_count,verified,average_tweets_per_day,account_age_days,account_type
0,0,2016-10-15 21:32:11,False,False,"Blame @xaiax, Inspired by @MakingInvisible, us...",4,1589,4,False,787405734442958848,en,unknown,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/7874121826...,best_in_dumbest,11041,False,7.870,1403,bot
1,1,2016-11-09 05:01:30,False,False,Photographing the American West since 1980. I ...,536,860,880,False,796216118331310080,en,Estados Unidos,http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/8023296328...,CJRubinPhoto,252,False,0.183,1379,human
2,2,2017-06-17 05:34:27,False,False,Scruffy looking nerf herder and @twitch broadc...,3307,172,594,True,875949740503859204,en,"Los Angeles, CA",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/1278890453...,SVGEGENT,1001,False,0.864,1159,human
3,3,2016-07-21 13:32:25,True,False,Wife.Godmother.Friend.Feline Fanatic! Assistan...,8433,517,633,True,756119643622735875,en,"Birmingham, AL",NaN,http://pbs.twimg.com/profile_images/1284884924...,TinkerVHELPK5,1324,False,0.889,1489,human
4,4,2012-01-15 16:32:35,False,False,Loan coach at @mancity & Aspiring DJ,88,753678,116,True,464781334,en,"England, United Kingdom",http://abs.twimg.com/images/themes/theme1/bg.png,http://pbs.twimg.com/profile_images/9952566258...,JoleonLescott,4202,True,1.339,3138,human


In [2]:
# Check cuda status
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [3]:
# Load pre-trained model and tokenizer
model_name = "bert-base-uncased"  # You can choose other models like "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Initialize pipeline
bot_detection_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, device=device)

# Sample Twitter descriptions
descriptions = [
    "I love this!",
    "Buy followers, likes, and retweets!",
    "I'm so happy.",
    "Check out this amazing deal!",
    "It's okay.",
    "Absolutely fantastic!",
    "Not good at all."
]

# Analyze descriptions
for description in descriptions:
    result = bot_detection_pipeline(description)
    print(f"Description: {description}")
    print(f"Prediction: {result}")
    print()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Description: I love this!
Prediction: [{'label': 'LABEL_1', 'score': 0.46820929646492004}]

Description: Buy followers, likes, and retweets!
Prediction: [{'label': 'LABEL_1', 'score': 0.4000426232814789}]

Description: I'm so happy.
Prediction: [{'label': 'LABEL_1', 'score': 0.43870222568511963}]

Description: Check out this amazing deal!
Prediction: [{'label': 'LABEL_1', 'score': 0.46263229846954346}]

Description: It's okay.
Prediction: [{'label': 'LABEL_1', 'score': 0.42986100912094116}]

Description: Absolutely fantastic!
Prediction: [{'label': 'LABEL_1', 'score': 0.4574173092842102}]

Description: Not good at all.
Prediction: [{'label': 'LABEL_1', 'score': 0.44880273938179016}]



In [9]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input': self.encodings.iloc[idx],
            'label': self.labels.iloc[idx]
        }

    def __len__(self):
        return len(self.labels)

In [12]:
# Split the data into training and evaluation datasets
X = df["description"]
y = df["account_type"]

X_train, y_train, X_eval, y_eval = train_test_split(X, y, test_size=0.2, random_state=42)
train_dataset = CustomDataset(X_train, y_train)
eval_dataset = CustomDataset(X_eval, y_eval)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Fine-tune the model
trainer.train()

AttributeError: 'CustomDataset' object has no attribute '_data'